**Analysis of optimalization algorithms for Softmax Classification Neural Network (Multi-layer perceptron)**

This model is designed to classify attributes of the patterns / pattern languages or sequences to a set of the classes and serves better than high-level API in Keras / Tensorflow, because we can better explain what our neural network is basically doing. Class with a highest probability is a prediction for a given sample in training dataset.

Analysis results:
- best optimizer so far is Adam devised in this work: https://arxiv.org/pdf/1412.6980.pdf
- RMSprop is second
- SGD is last but can be enhanced with SGDMomentum

Because dataset of the attributes is still being consulted, sample dataset with 11 independent variables is provided. Dependent variable is a label, class. There are 10 classes in a dataset, classes 5, 6 and 7 are most present (some patterns are used more often than the others).

Currently these options are being considered:

- term frequencies (tf)
- inverse frequencies (tf-idf)
- probabilities (from our work Modelling of Organizational Pattern Sequences in Bayesian Network)
- Table 1 from section 3.3. in http://www2.fiit.stuba.sk/~vranic/pub/ExtractingRelations.pdf

Other unresolved (implementation) problems:

- current implementation fails to work with categorical entropy and 10 output neurons, but works well with sparse categorical crossentropy. Usual cause for this is that dependent variable (pattern sequence encoded with number) is not one-hot encoded. See what one-hot encoding means.
- 2 hidden layers, one of them with 350 neurons is still a lot for this small dataset
- work with smaller batches

<ins>Example how to interpret output from this Neural network</ins>:

Let's say prediction for a first row in our training dataset is a vector of values: (0.08568677, 0.09945365, 0.08751229, 0.09474804, 0.1098659 , 0.12171782, 0.10679027, 0.10450635, 0.10343555, 0.08628327). This means first row in dataset has been assigned to class 6 because of its highest probability (0.1211782). Class 6 can represent organizational pattern, organizational pattern language or sequence of organizational patterns.

Please note that prediction is correct if it is consistent with actual class. Accuracy of the model is one of the metrics used to evaluate this behavior.

Theory behind this can be found in a book Dive into Deep Learning: https://d2l.ai/chapter_linear-networks/softmax-regression.html

In [ ]:
import numpy as np
from utils import Module
import numpy as np
import pandas as pd

In [ ]:
frequencies = pd.read_csv('dataset.csv', sep = ';')
train, val, test = np.split(frequencies.sample(frac=1, random_state=42), [int(.6*len(frequencies)), int(.8*len(frequencies))])

class Linear(Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        # vahy by nemali byt nahodne ale 1/n nasobok vah co pomaha stabilizacii ucenia
        self.W = np.random.randn(out_features, in_features)
        self.b = np.zeros((out_features, 1))

    def forward(self, input: np.ndarray) -> np.ndarray:
        self.aPred = input # aktivacia predchadzajucej vrstvy je sucastou cache pre spatny prechod
        self.m = self.aPred.shape[0] # pocet vzoriek je tiez sucastou cache pre spatny prechod
        net = np.matmul(self.W, input) + self.b
        return net

    def backward(self, dz: np.ndarray) -> np.ndarray:
        # dW a db su gradienty. dw udava smer narastania chyb. Pri zostupe potrebujeme ist proti smeru gradientu
        self.dW = (1.0/self.m) * np.sum(np.matmul(dz, self.aPred.transpose((0,2,1))), axis=0) # aktivaciu si vrstva drzi aj pri spatnom prechode
        self.db = (1.0/self.m) * np.sum(dz, axis=0)
        # vrat vstup pre dalsi spatny prechod dalsou vrstvou. Ten sa vynasobi derivaciou aktivacnej funkcie
        return np.matmul(self.W.transpose(), dz) # alebo return self.W.T @ dz, vysledkom bude vektor, spatne sirenie

    def get_optimizer_context(self):
        return [[self.W, self.dW], [self.b, self.db]]

    def update_parameters(self, params): # commit 4ddc896096c5b12af45b0328ac399de5be54b88e
        self.W, self.b = params
        
#------------------------------------------------------------------------------
#   SigmoidActivationFunction class
#------------------------------------------------------------------------------
class Sigmoid(Module):
    def __init__(self):
        super(Sigmoid, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        self.fw_input = input
        return 1.0 / (1.0 + np.exp(-input))

    def backward(self, da) -> np.ndarray:
        a = self(self.fw_input)
        return np.multiply(da, np.multiply(a, 1 - a))

#------------------------------------------------------------------------------
#   HyperbolicTangentActivationFunction class
#------------------------------------------------------------------------------
class Tanh(Module):
    def __init__(self):
        super(Tanh, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        self.fw_input = input
        return (np.exp(2 * input) - 1) / (np.exp(2 * input) + 1)

    def backward(self, da) -> np.ndarray:
        a = self(self.fw_input)
        return np.multiply(da, 1 - np.square(a))

#------------------------------------------------------------------------------
#   Model class
#------------------------------------------------------------------------------
class Model(Module):
    def __init__(self):
        super(Model, self).__init__()

    def forward(self, input) -> np.ndarray:
        for name, module in self.modules.items():
            input = module(input)
        return input

    def backward(self, z: np.ndarray):
        for name, module in reversed(self.modules.items()):
            z = module.backward(z)

In [ ]:
#------------------------------------------------------------------------------
#   MeanSquareErrorLossFunction class
#------------------------------------------------------------------------------
class MSELoss(Module):
    def __init__(self, reduce="mean"):
        super(MSELoss, self).__init__()
        if reduce == "mean":
            self.reduce_fn = np.mean
        elif reduce == "sum":
            self.reduce_fn = np.sum
        elif reduce is None:
            # return identity (do nothing)
            self.reduce_fn = lambda x : x
        else:
            raise AttributeError

    def forward(self, input: np.ndarray, target: np.ndarray) -> np.ndarray:
        return self.reduce_fn(np.mean(np.power(target - input, 2), axis=0, keepdims=True))

    def backward(self, input: np.ndarray, target: np.ndarray) -> np.ndarray:
        return np.mean(-2 * (target - input), axis=1, keepdims=True)


#------------------------------------------------------------------------------
#   BinaryCrossEntropyLossFunction class
#------------------------------------------------------------------------------
class BCELoss(Module):
    def __init__(self, reduce="mean"):
        super(BCELoss, self).__init__()
        if reduce == "mean":
            self.reduce_fn = np.mean
        elif reduce == "sum":
            self.reduce_fn = np.sum
        elif reduce is None:
            # return identity (do nothing)
            self.reduce_fn = lambda x : x
        else:
            raise AttributeError

    def forward(self, input: np.ndarray, target: np.ndarray) -> np.ndarray:
        # toto je v podstate mozne napisat aj ako
        # - (np.multiply(target, np.log(input)) + np.multiply((1-target, np.log(1-input)))
        return self.reduce_fn(-(target * np.log(input) + np.multiply((1 - target), np.log(1 - input))))

    def backward(self, input: np.ndarray, target: np.ndarray) -> np.ndarray:
        # derivacia loss funkcie podla aktivacie je -y/a + (1-y)/(1-a) kde y je target a "a" je input
        # spatny prechod podla 2. prednasky v tomto semestri
        return -np.divide(target, input) + np.divide(1 - target, 1 - input)

In [ ]:
#------------------------------------------------------------------------------
#   AbstractOptimizer class
#------------------------------------------------------------------------------
class Optimizer:
    def __init__(self):
        pass

    def step(self):
        raise NotImplemented

#------------------------------------------------------------------------------
# StochasticGradientDescentOptimizer class
# objekt tejto triedy realizuje jeden krok zostupu
# zostup robi pri kazdom riadku datasetu
# tento optimalizator je nevhodny pre vela prechodov / epoch
# riesenim je nasekat dataset na davky
#------------------------------------------------------------------------------
class SGD(Optimizer):
    def __init__(self, model:Model, lr:float):
        super(SGD, self).__init__()
        self.model = model
        self.lr = lr
        self.context = {}

    def step(self):
        for name, layer in self.model.modules.items():
            if hasattr(layer, 'get_optimizer_context'):
                params = layer.get_optimizer_context()
                if params is not None:
                    [[W, dW],[b,db]] = params
                    W = W - self.lr * dW # povodna hodnota - nasobok rychlosti ucenia*gradient
                    b = b - self.lr * db
                    layer.update_parameters([W,b])


In [ ]:
#------------------------------------------------------------------------------
# SGDMomentumOptimizer class
# ma tendenciu ist priamociarejsie ist k minimu ako SGD hore
#------------------------------------------------------------------------------
class SGDMomentum(Optimizer):
    def __init__(self, model, lr, beta): #dopln svoje
        super(SGDMomentum, self).__init__()
        self.model = model
        self.context = {}
        # >>>> start_solution
        self.lr = lr
        self.beta = beta
        # <<<< end_solution

    def step(self):
        for name, layer in self.model.modules.items():
            if hasattr(layer, 'get_optimizer_context'):
                params = layer.get_optimizer_context()
                if params is not None:
                    [[W, dW],[b,db]] = params
                    if name in self.context.keys(): #dictionary pre rychlosti pravdepodobne, ak uz mas rychlost tak ju pouzi, inak vytvor
                    # >>>> start_solution
                        self.context[name]["Vdw"] = (1 - self.beta) * dW + self.beta * self.context[name]["Vdw"]
                        self.context[name]["Vdb"] = (1 - self.beta) * db + self.beta * self.context[name]["Vdb"]
                        #pass
                    else:#nie su rychlosti v kontexte, takze su nula
                        self.context[name] = {}
                        self.context[name]["Vdw"] = (1 - self.beta) * dW
                        self.context[name]["Vdb"] = (1 - self.beta) * db
                        
                        #pass
                    W = W - self.lr*self.context[name]["Vdw"]
                    b = b - self.lr*self.context[name]["Vdb"]
                    # <<<< end_solution
                    layer.update_parameters([W,b])

#------------------------------------------------------------------------------
# RMSpropOptimizer class
# vznikol na prednaske Geoffreyho Hintona
# manipuluje s dlzkou kroku na ceste k optimalnym parametrom v zavislosti od velkosti gradientov
#------------------------------------------------------------------------------
class RMSprop(Optimizer): #podobne ako SGDMomentum
    def __init__(self, model, lr, beta, epsilon):
        super(RMSprop, self).__init__()
        self.model = model
        self.context = {}
        # >>>> start_solution
        self.lr = lr
        self.beta = beta
        self.epsilon = epsilon
        # <<<< end_solution

    def step(self):
        for name, layer in self.model.modules.items():
            if hasattr(layer, 'get_optimizer_context'):
                params = layer.get_optimizer_context()
                if params is not None:
                    [[W, dW], [b, db]] = params
                    if name in self.context.keys():
                        # vzorce zo slajdu c. 81 prednasky Lecture3_NN_improvements
                        self.context[name]["Sdw"] = (1 - self.beta) * np.square(dW) + self.beta * self.context[name]["Sdw"]
                        self.context[name]["Sdb"] = (1 - self.beta) * np.square(db) + self.beta * self.context[name]["Sdb"]
                        #pass
                    else:
                        self.context[name] = {}
                        self.context[name]["Sdw"] = (1 - self.beta) * np.square(dW)
                        self.context[name]["Sdb"] = (1 - self.beta) * np.square(db)
                        #pass
                        
                    # tam kde mame maly gradient zvacsime dlzku kroku
                    # tam kde mame velky gradient zmensime dlzku kroku
                    # pri kazdom vzostupe pocitame podiel gradientu vah a druhej odmocniny Sdw (pozri vzorec)
                    W = W - self.lr * (dW / (np.sqrt(self.context[name]["Sdw"]) + self.epsilon))
                    # pri kazdom vzostupe pocitame podiel gradientu biasu a druhej odmocniny Sdb (pozri vzorec)
                    b = b - self.lr * (db / (np.sqrt(self.context[name]["Sdb"]) + self.epsilon))
                    layer.update_parameters([W, b])


#------------------------------------------------------------------------------
# AdamOptimizer class
# algoritmus z knihy Deep Learning od Iana Goodfellowa et al.
#------------------------------------------------------------------------------
class Adam(Optimizer):
    def __init__(self, model, lr, beta1, beta2, epsilon):
        super(Adam, self).__init__()
        self.model = model
        self.context = {}
        self.beta1 = beta1
        self.beta2 = beta2
        self.lr = lr     
        self.epsilon = epsilon
        self.iteration = 0

    def step(self):
        # premenna iteration znamena timestamp v algoritme na str. 2 v praci Kingmu et al.
        # pozri: https://www.researchgate.net/publication/269935079_Adam_A_Method_for_Stochastic_Optimization
        self.iteration = self.iteration + 1
        # <<<<<< until here
        for name, layer in self.model.modules.items():
            if hasattr(layer, 'get_optimizer_context'):
                params = layer.get_optimizer_context()
                if params is not None:
                    [[W, dW], [b, db]] = params
                    if name in self.context.keys(): 
                        # >>>> start_solution
                        # first_moment_estimate = beta1*first_moment_estimate + (1-beta1)*gradientVah 
                        self.context[name]["Vdw"] = (1 - self.beta1) * dW + self.beta1 * self.context[name]["Vdw"]
                        # second_moment_estimate = beta2*second_moment_estimate + (1-beta2)*gradientVah**2
                        self.context[name]["Sdw"] = (1 - self.beta2) * np.square(dW) + self.beta2 * self.context[name]["Sdw"]
                        # first_moment_estimate = beta1*first_moment_estimate + (1-beta1)*gradientBiasu
                        self.context[name]["Vdb"] = (1 - self.beta1) * db + self.beta1 * self.context[name]["Vdb"]
                        # second_moment_estimate = beta2*second_moment_estimate + (1-beta2)*gradientBiasu**2
                        self.context[name]["Sdb"] = (1 - self.beta2) * np.square(db) + self.beta2 * self.context[name]["Sdb"]
                        #pass
                    else:
                        self.context[name] = {}
                        self.context[name]["Vdw"] = (1 - self.beta1) * dW
                        self.context[name]["Sdw"] = (1 - self.beta2) * np.square(dW)

                        self.context[name]["Vdb"] = (1 - self.beta1) * db
                        self.context[name]["Sdb"] = (1 - self.beta2) * np.square(db)
                        #pass
                    # compute bias-corrected first and second raw moment estimates    
                    Vdw_corr = self.context[name]["Vdw"] / (1 - self.beta1**self.iteration)
                    Vdb_corr = self.context[name]["Vdb"] / (1 - self.beta1**self.iteration)
                    Sdw_corr = self.context[name]["Sdw"] / (1 - self.beta2**self.iteration)
                    Sdb_corr = self.context[name]["Sdb"] / (1 - self.beta2**self.iteration)
                    
                    W = W - self.lr * (Vdw_corr / (np.sqrt(Sdw_corr) + self.epsilon))
                    b = b - self.lr * (Vdb_corr / (np.sqrt(Sdb_corr) + self.epsilon))
                    # <<<< end_solution
                    layer.update_parameters([W, b])



In [ ]:
from utils import gradient_check
loss_dict = {}

In [ ]:
mlpForSGD = Model()
mlpForSGD.add_module(Linear(2, 3), 'Dense_1')
mlpForSGD.add_module(Tanh(), 'Tanh_1')
mlpForSGD.add_module(Linear(3, 4), 'Dense_2')
mlpForSGD.add_module(Tanh(), 'Tanh_2')
mlpForSGD.add_module(Linear(4, 5), 'Dense_3')
mlpForSGD.add_module(Tanh(), 'Tanh_3')
mlpForSGD.add_module(Linear(5, 1), 'Dense_4_out')
mlpForSGD.add_module(Sigmoid(), 'Sigmoid') # najcastejsie je sigmoid posledna vrstva
loss_fn = MSELoss(reduce='mean')

mlpForAdam = Model()
mlpForAdam.add_module(Linear(2, 3), 'Dense_1')
mlpForAdam.add_module(Tanh(), 'Tanh_1')
mlpForAdam.add_module(Linear(3, 4), 'Dense_2')
mlpForAdam.add_module(Tanh(), 'Tanh_2')
mlpForAdam.add_module(Linear(4, 5), 'Dense_3')
mlpForAdam.add_module(Tanh(), 'Tanh_3')
mlpForAdam.add_module(Linear(5, 1), 'Dense_4_out')
mlpForAdam.add_module(Sigmoid(), 'Sigmoid')

mlpForSGDMomentum = Model()
mlpForSGDMomentum.add_module(Linear(2, 3), 'Dense_1')
mlpForSGDMomentum.add_module(Tanh(), 'Tanh_1')
mlpForSGDMomentum.add_module(Linear(3, 4), 'Dense_2')
mlpForSGDMomentum.add_module(Tanh(), 'Tanh_2')
mlpForSGDMomentum.add_module(Linear(4, 5), 'Dense_3')
mlpForSGDMomentum.add_module(Tanh(), 'Tanh_3')
mlpForSGDMomentum.add_module(Linear(5, 1), 'Dense_4_out')
mlpForSGDMomentum.add_module(Sigmoid(), 'Sigmoid')

mlpForRMSProp = Model()
mlpForRMSProp.add_module(Linear(2, 3), 'Dense_1')
mlpForRMSProp.add_module(Tanh(), 'Tanh_1')
mlpForRMSProp.add_module(Linear(3, 4), 'Dense_2')
mlpForRMSProp.add_module(Tanh(), 'Tanh_2')
mlpForRMSProp.add_module(Linear(4, 5), 'Dense_3')
mlpForRMSProp.add_module(Tanh(), 'Tanh_3')
mlpForRMSProp.add_module(Linear(5, 1), 'Dense_4_out')
mlpForRMSProp.add_module(Sigmoid(), 'Sigmoid')

optimizerAdam = Adam(mlpForAdam, lr=0.001, beta1 = 0.9, beta2 = 0.999, epsilon = 10**(-8))

N_epochs = 1000
lossesForAdam = []
for i in range(N_epochs):
    epoch_loss = []
    for mini_batch_X, mini_batch_Y in dataset:
        # priamy prechod
        predicted_Y_hat = mlpForAdam.forward(mini_batch_X) # predikcia
        loss = loss_fn(predicted_Y_hat, mini_batch_Y) # strata pre kazdu vzorku
        epoch_loss += [np.mean(loss)] # naklad pre kazdy mini batch
        gradients = loss_fn.backward(predicted_Y_hat, mini_batch_Y)
        # spatny prechod
        mlpForAdam.backward(gradients)
        # gradient_check(mlpForAdam, loss_fn, mini_batch_X, mini_batch_Y)
        optimizerAdam.step()
    lossesForAdam += [np.mean(epoch_loss)]
    
loss_dict['Adam'] = lossesForAdam

optimizerSGD = SGD(mlpForSGD, lr=0.001)

N_epochs = 1000
lossesForSGD = []
for i in range(N_epochs):
    epoch_loss = []
    for mini_batch_X, mini_batch_Y in dataset:
        # priamy prechod
        predicted_Y_hat = mlpForSGD.forward(mini_batch_X) # predikcia
        loss = loss_fn(predicted_Y_hat, mini_batch_Y) # strata pre kazdu vzorku
        epoch_loss += [np.mean(loss)] # naklad pre kazdy mini batch
        gradients = loss_fn.backward(predicted_Y_hat, mini_batch_Y)
        # spatny prechod
        mlpForSGD.backward(gradients)
        # gradient_check(mlpForSGD, loss_fn, mini_batch_X, mini_batch_Y)
        optimizerSGD.step()
    lossesForSGD += [np.mean(epoch_loss)]
    
loss_dict['SGD'] = lossesForSGD

optimizerSGDMomentum = SGDMomentum(mlpForSGDMomentum, lr=0.001, beta=0.9)
N_epochs = 1000
lossesForSGDMomentum = []
for i in range(N_epochs):
    epoch_loss = []
    for mini_batch_X, mini_batch_Y in dataset:
        predicted_Y_hat = mlpForSGDMomentum.forward(mini_batch_X)
        loss = loss_fn(predicted_Y_hat, mini_batch_Y)
        epoch_loss += [np.mean(loss)]
        gradients = loss_fn.backward(predicted_Y_hat, mini_batch_Y)
        mlpForSGDMomentum.backward(gradients)
        # gradient_check(mlpForSGDMomentum, loss_fn, mini_batch_X, mini_batch_Y)
        optimizerSGDMomentum.step()
    lossesForSGDMomentum += [np.mean(epoch_loss)]
    
loss_dict['SGDMomentum'] = lossesForSGDMomentum

optimizerRMSProp = RMSprop(mlpForRMSProp, lr=0.001, beta=0.9, epsilon = 10**(-8))
N_epochs = 1000
lossesForRMSProp = []
for i in range(N_epochs):
    epoch_loss = []
    for mini_batch_X, mini_batch_Y in dataset:
        predicted_Y_hat = mlpForRMSProp.forward(mini_batch_X)
        loss = loss_fn(predicted_Y_hat, mini_batch_Y)
        epoch_loss += [np.mean(loss)]
        gradients = loss_fn.backward(predicted_Y_hat, mini_batch_Y)
        mlpForRMSProp.backward(gradients)
        # gradient_check(mlpForRMSProp, loss_fn, mini_batch_X, mini_batch_Y)
        optimizerRMSProp.step()
    lossesForRMSProp += [np.mean(epoch_loss)]
    
loss_dict['RMSprop'] = lossesForRMSProp